# 🧹 02 - Data Cleaning: Telco Customer Churn

Notebook ini bertujuan untuk membersihkan dataset, menangani missing values, dan mempersiapkan fitur agar siap untuk proses analisis lanjutan.

## 1. Import Library & Load Dataset

Dataset yang digunakan berasal dari file yang telah dieksplorasi sebelumnya di tahap Data Understanding.


In [50]:
import os
import pandas as pd

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_raw_path = os.path.join(project_root, 'data', 'raw')

df = pd.read_csv('../data/raw/telco_raw.csv')

## 2 Drop Kolom Yang Tidak Dibutuhkan

Kolom 'customerID' merupakan identifier unik dan tidak memiliki nilai prediktif, sehingga akan dihapus.

In [51]:
df.drop(columns=['customerID'], inplace=True)

## 3 Menyesuaikan Value Pada Kolom TotalCharges

Kolom `TotalCharges` awalnya terbaca sebagai object/string karena adanya karakter non-numerik atau missing value. Kita konversi ke tipe float agar bisa digunakan dalam analisis numerik.

In [52]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

print("Missing values setelah konversi TotalCharges:", df['TotalCharges'].isnull().sum())

Missing values setelah konversi TotalCharges: 11


Kolom `TotalCharges` mengandung nilai kosong setelah dikonversi ke float. Terjadinya missing value dikarenakan isi pada kolom `Tenure` = 0. Oleh karena itu kita akan menyesuaikan isi dari `TotalCharges` menjadi sama dengan isi dari kolom `MonthlyCharges`.

In [53]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

df['TotalCharges'] = df['TotalCharges'].fillna(df['MonthlyCharges'])

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   object 
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   object 
 3   Dependents        7043 non-null   object 
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   object 
 6   MultipleLines     7043 non-null   object 
 7   InternetService   7043 non-null   object 
 8   OnlineSecurity    7043 non-null   object 
 9   OnlineBackup      7043 non-null   object 
 10  DeviceProtection  7043 non-null   object 
 11  TechSupport       7043 non-null   object 
 12  StreamingTV       7043 non-null   object 
 13  StreamingMovies   7043 non-null   object 
 14  Contract          7043 non-null   object 
 15  PaperlessBilling  7043 non-null   object 
 16  PaymentMethod     7043 non-null   object 


## 4 Konversi Tipe Data

Kolom `SeniorCitizen` perlu diubah ke tipe kategorikal.

In [54]:
df['SeniorCitizen'] = df['SeniorCitizen'].astype('object')

## 5. Cek Kategori & Normalisasi Nilai

Beberapa nilai kategori bisa punya typo atau format inkonsisten. Kita pastikan semua kategori sudah rapi.

In [55]:
for col in df.select_dtypes(include='object').columns:
    print(f"{col}: {df[col].unique()}")

gender: ['Female' 'Male']
SeniorCitizen: [0 1]
Partner: ['Yes' 'No']
Dependents: ['No' 'Yes']
PhoneService: ['No' 'Yes']
MultipleLines: ['No phone service' 'No' 'Yes']
InternetService: ['DSL' 'Fiber optic' 'No']
OnlineSecurity: ['No' 'Yes' 'No internet service']
OnlineBackup: ['Yes' 'No' 'No internet service']
DeviceProtection: ['No' 'Yes' 'No internet service']
TechSupport: ['No' 'Yes' 'No internet service']
StreamingTV: ['No' 'Yes' 'No internet service']
StreamingMovies: ['No' 'Yes' 'No internet service']
Contract: ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: ['Yes' 'No']
PaymentMethod: ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: ['No' 'Yes']


Menghilangkan redundansi data (seperti `No phone service` dan `No internet service` ) pada kolom `MultipleLines`, `OnlineSecurity`, `OnlineBackup`, `DeviceProtection`, `TechSupport`, `StreamingTV`, dan `StreamingMovies`.

In [56]:
df.replace('No phone service', 'No', inplace=True)
df.replace('No internet service', 'No', inplace=True)

In [57]:
df.isnull().sum()

gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

## 6. Simpan Dataset yang Sudah Dibersihkan

Dataset hasil cleaning disimpan ke dalam folder `data/cleaned/` untuk digunakan di tahap analisis selanjutnya.

In [58]:
df.to_csv("../data/processed/telco_cleaned.csv", index=False)


## 📝 7. Ringkasan Cleaning

- Kolom `customerID` dihapus
- Missing value pada `TotalCharges` ditangani
- `SeniorCitizen` dikonversi ke kategorikal
- Normalisasi nilai kategori seperti `'No internet service'` → `'No'`